# Lesson 7：Python `socket` 网络编程入门

`socket` 模块提供底层网络通信能力，可创建 TCP/UDP 客户端与服务端。本课聚焦常见用法，帮助快速构建本地测试服务或理解高层框架的基础原理。

## 1. 核心概念回顾

- **IP 地址**：标识网络设备的位置（IPv4/IPv6）。
- **端口（Port）**：在同一主机上区分不同服务，范围 0-65535。
- **套接字（Socket）**：应用程序与网络之间的双向通信端点。
- **协议族**：`AF_INET`（IPv4）、`AF_INET6`（IPv6）、`AF_UNIX`（同机器通信）。
- **套接字类型**：`SOCK_STREAM`（TCP，可靠且面向连接）、`SOCK_DGRAM`（UDP，无连接）。

## 2. 获取主机信息

In [ ]:
# socket 提供多种工具查询主机与地址信息
import socket

print("当前主机名: ", socket.gethostname())
print("本机回环地址(IPv4):", socket.gethostbyname('localhost'))

# getaddrinfo 返回列表，包含可用地址族、套接字类型等信息
info = socket.getaddrinfo('localhost', 8080, proto=socket.IPPROTO_TCP)
print("可用地址组合数量:", len(info))
for family, socktype, proto, canonname, sockaddr in info[:2]:
    print({
        'family': family,
        'socktype': socktype,
        'proto': proto,
        'sockaddr': sockaddr,
    })


## 3. 使用 `socketpair` 演示基本收发

`socket.socketpair()` 创建成对的套接字，适合在同一进程内模拟双向通信，便于教学与测试。

In [ ]:
# 在同一进程内创建一对连接的套接字
import socket

left, right = socket.socketpair()
try:
    left.sendall(b"hello")
    data = right.recv(1024)
    print("右侧接收到:", data)

    right.sendall(b"world")
    print("左侧接收到:", left.recv(1024))
finally:
    left.close()
    right.close()


## 4. 构建最小 TCP 服务端与客户端

以下示例展示如何在本地创建一个简单的 Echo 服务。

### 注意
- 在 Notebook 中运行时需保证端口未被占用。
- 服务端通常在单独的线程或进程运行。
- 演示使用 `127.0.0.1`（本地回环地址），避免网络权限问题。


In [ ]:
# 简易 TCP Echo 服务示例（服务器 + 客户端）
import socket
import threading

HOST = '127.0.0.1'
PORT = 50500


def handle_client(conn, addr):
    with conn:
        print(f"客户端连接: {addr}")
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)
        print(f"客户端断开: {addr}")


def start_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server:
        server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server.bind((HOST, PORT))
        server.listen()
        print(f"Echo 服务监听 {HOST}:{PORT}")
        while True:
            conn, addr = server.accept()
            thread = threading.Thread(target=handle_client, args=(conn, addr), daemon=True)
            thread.start()


server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# 客户端演示
with socket.create_connection((HOST, PORT)) as client:
    client.sendall(b"hello socket")
    response = client.recv(1024)
    print("客户端收到:", response.decode())


## 5. UDP 通信示例

UDP 为无连接协议，适合实时性要求高而允许丢包的场景（如影音流、简单心跳）。示例展示如何发送与接收数据报。

In [ ]:
# UDP Send/Receive 示例（同进程模拟）
import socket

SERVER_ADDR = ('127.0.0.1', 50600)

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as server:
    server.bind(SERVER_ADDR)
    server.settimeout(1)

    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as client:
        client.sendto(b"ping", SERVER_ADDR)
        try:
            data, address = server.recvfrom(1024)
            print("服务器收到:", data, "来自", address)
            server.sendto(b"pong", address)
        except socket.timeout:
            print("服务器等待超时")

        client.settimeout(1)
        try:
            reply, _ = client.recvfrom(1024)
            print("客户端收到:", reply)
        except socket.timeout:
            print("客户端等待超时")


### 5.1 socket 版 TCP Echo（同步）与 UDP Echo

下面通过两个最小示例，展示如何使用 `socket` 创建同步的 TCP Echo 服务端/客户端以及一个单往返的 UDP Echo。示例使用大量注释强调关键步骤，便于教学。

In [ ]:
# --- 同步 TCP Echo 示例 ---
import socket

HOST = '127.0.0.1'
PORT = 50700

# 1) 构建 TCP 套接字并绑定到本地端口。
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind((HOST, PORT))
server.listen(1)  # 同步示例，仅接受一个连接即可
print(f"[TCP] 服务器监听 {HOST}:{PORT}")

# 2) 客户端建立连接并发送消息。
client = socket.create_connection((HOST, PORT))
client.sendall(b"hello tcp\n")

# 3) 服务器 accept 连接，这一步会阻塞直到有客户端到来。
conn, addr = server.accept()
print(f"[TCP] 服务器收到连接: {addr}")

# 4) 服务器读取数据后原样返回，实现 Echo。
data = conn.recv(1024)
print(f"[TCP] 服务器收到: {data!r}")
conn.sendall(data)

# 5) 客户端读取服务器返回的内容。
reply = client.recv(1024)
print(f"[TCP] 客户端收到: {reply!r}")

# 6) 关闭双方连接与监听套接字，释放资源。
conn.close()
client.close()
server.close()

# --- UDP Echo 示例 ---

# UDP 无连接，双方使用 sendto / recvfrom 即可。
udp_server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
udp_server.bind((HOST, PORT + 1))
print(f"[UDP] 服务器绑定 {HOST}:{PORT + 1}")

udp_client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
udp_message = b"hello udp"
udp_client.sendto(udp_message, (HOST, PORT + 1))
print(f"[UDP] 客户端发送: {udp_message!r}")

received, client_addr = udp_server.recvfrom(1024)
print(f"[UDP] 服务器收到: {received!r} 来自 {client_addr}")
udp_server.sendto(received, client_addr)

udp_echo, _ = udp_client.recvfrom(1024)
print(f"[UDP] 客户端收到: {udp_echo!r}")

udp_client.close()
udp_server.close()


## 6. 套接字选项与超时

- 使用 `setdefaulttimeout` 或 `socket.settimeout` 控制阻塞时间。
- `setsockopt` 可开启 `SO_REUSEADDR`、`TCP_NODELAY` 等高级特性。

In [ ]:
# 设置超时时间示例
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(0.5)  # 秒
print("当前超时设置:", sock.gettimeout())

# 调整 TCP_NODELAY（禁用 Nagle 算法）
sock.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
print("TCP_NODELAY 设置完成")
sock.close()


## 7. 资源清理与上下文管理

- 使用 `with socket.socket(...) as sock:` 保证退出时自动关闭。
- 在多线程环境中结合 `daemon=True` 或显式 join，避免资源泄漏。

In [ ]:
# 使用上下文管理器确保关闭套接字
import socket

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as temp_sock:
    temp_sock.bind(('127.0.0.1', 0))  # 端口 0 让系统自动分配临时端口
    addr = temp_sock.getsockname()
    print("临时监听地址:", addr)

print("超出 with 语句后套接字自动关闭")


## 8. 常见调试技巧

- 使用 `nc`（netcat）或 `telnet` 与自写服务交互。
- `tcpdump`/`Wireshark` 捕获报文分析协议细节。
- 通过 `socket.getsockname()` 与 `getpeername()` 查看本端/远端地址。
- 开启日志记录，输出关键事件与异常信息便于定位问题。

## 9. 非阻塞模式与 `setblocking`

- 默认情况下套接字是阻塞的，`recv`/`send` 会等待直到有结果。
- 使用 `setblocking(False)` 或 `settimeout(0)` 可切换为非阻塞模式；无数据时会抛出 `BlockingIOError`。
- 结合非阻塞 I/O 可以构建自己的事件循环或与 `select`/`selectors` 配合。

In [ ]:
# 使用 socketpair 演示阻塞与非阻塞行为
import socket

left, right = socket.socketpair()
try:
    left.setblocking(False)
    try:
        left.recv(1)
    except BlockingIOError as exc:
        print("非阻塞套接字在无数据时抛出:", exc)

    # 发送数据后再尝试读取
    right.sendall(b"data")
    print("读取到:", left.recv(4))
finally:
    left.close()
    right.close()


## 10. 使用 `selectors` 管理多个连接

`selectors` 模块封装了 `select`/`poll` 等系统调用，可在单线程内同时处理多个连接：

In [ ]:
# 使用 selectors 构建简化的 echo 循环
import selectors
import socket

sel = selectors.DefaultSelector()
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind(('127.0.0.1', 0))
server.listen()
server.setblocking(False)
sel.register(server, selectors.EVENT_READ)

server_host, server_port = server.getsockname()

client = socket.create_connection((server_host, server_port))
client.sendall(b'hello from selectors')
client.shutdown(socket.SHUT_WR)

done = False
while not done:
    for key, mask in sel.select(timeout=1):
        if key.fileobj is server:
            conn, addr = server.accept()
            conn.setblocking(False)
            sel.register(conn, selectors.EVENT_READ, bytearray())
        else:
            conn = key.fileobj
            buffer = key.data
            chunk = conn.recv(1024)
            if chunk:
                buffer.extend(chunk)
                conn.sendall(b'ECHO:' + chunk)
            else:
                print('服务端收到:', buffer.decode())
                sel.unregister(conn)
                conn.close()
                done = True
    if not sel.get_map():
        break

print('客户端收到:', client.recv(1024).decode())
client.close()
sel.unregister(server)
server.close()
sel.close()


## 11. 将套接字当作文件对象

- `socket.makefile()` 返回类文件对象，方便使用 `readline()`、`read()` 等方法处理文本协议。
- 标准库 `socketserver`、`http.server` 等高层框架内部也大量使用 `makefile`。

In [ ]:
# 使用 makefile 将 socket 抽象为文件对象
import socket

left, right = socket.socketpair()
try:
    with left.makefile('wb') as writer, right.makefile('rb') as reader:
        writer.write(b'first line second line')
        writer.flush()
        print('读取 1:', reader.readline().decode().strip())
        print('读取 2:', reader.readline().decode().strip())
finally:
    left.close()
    right.close()


## 12. TLS/SSL 安全连接概览

- 使用 `ssl` 模块可以在套接字之上构建加密通道。
- 推荐通过 `ssl.create_default_context()` 创建安全的默认配置，并在客户端调用 `wrap_socket` 或 `SSLContext.wrap_socket`。
- 下面示例展示客户端与服务端的基本包装方法（演示代码不会主动连往互联网，运行需自行准备 TLS 服务端或证书）。

In [2]:
# TLS 客户端与服务端包装示例（演示用途）
import socket
import ssl

context = ssl.create_default_context()

# 作为客户端连接安全服务
def https_get(hostname: str, port: int = 443) -> bytes:
    with socket.create_connection((hostname, port)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as tls_sock:
            request = f"GET / HTTP/1.1 Host: {hostname} Connection: close".encode()
            tls_sock.sendall(request)
            chunks = []
            while True:
                data = tls_sock.recv(4096)
                if not data:
                    break
                chunks.append(data)
    return b''.join(chunks)

# 在本地构建 TLS 服务端时，可使用自签证书并通过 SSLContext.wrap_socket(server_socket, server_side=True)


## 13. 小结与练习建议
- 熟悉 TCP/UDP 创建流程：`socket` → `bind` → `listen/accept` 或 `connect`。
- 合理设置 `timeout`、`SO_REUSEADDR`、`TCP_NODELAY` 等选项，理解阻塞与非阻塞差异。
- 使用 `selectors`/`asyncio` 提升并发处理效率，必要时搭配 `makefile` 简化文本协议处理。
- 练习：编写一个服务端，将客户端发送的 JSON 消息解析后返回格式化响应。
- 练习：实现简单聊天室，使用多线程或 `selectors` 管理多个客户端，并尝试加上 TLS 加密。
- 推荐进一步了解 `socketserver`、`asyncio.Streams`、`trio` 等高层框架。